In [3]:
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json(r'C:\Users\amand\project\RA_WORK_CONTENT_BQ.json')
%load_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r'C:\Users\amand\project\RA_WORK_CONTENT_BQ.json'

import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')
table = '[ra-work-content.ceo_tweet_streaming.search_2017_14]'
import pandas_gbq


The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [5]:
sql_query="""
select status_id, fulltext
from %s
where is_retweet is false
group by status_id, fulltext
"""%(table)
query_config = bigquery.QueryJobConfig(use_legacy_sql=True)
ceo_tweet = client.query(sql_query, job_config=query_config).to_dataframe()
ceo_tweet.head(10)

,status_id,fulltext
0,1115342985657495552,"""Advocates for human rights and free speech wa..."
1,1043171266008625152,"""Either consumers will pay more, suppliers wil..."
2,1059830148483166209,"""Every dollar invested in sanitation yields ab..."
3,1110242701692289024,#DareToServe is a book for practitioners—peopl...
4,1021866865935740928,2008 was a seminal year for @Nasdaq. Following...
5,1118750312838455297,@BetoORourke has disqualified himself as a ser...
6,1075458012838547456,@CesarFloresTV I'm sorry to hear about your ex...
7,1112691118100099073,@HowardSchultz is by far the most qualified ca...
8,968518538859171845,@LouMM Here are some clues of companies only t...
9,936633471925215233,@MageeRehab helps improve the quality of life ...


In [6]:
sid = SentimentIntensityAnalyzer()


listy1 = []
listy2 = []
listy3 = []
listy4 = []



for index, row in ceo_tweet.iterrows():
  ss = sid.polarity_scores(row["fulltext"])
  listy1.append(ss['neg'])
  listy2.append(ss['neu'])
  listy3.append(ss['pos'])
  listy4.append(ss['compound'])

  
se1 = pd.Series(listy1)
se2 = pd.Series(listy2)
se3 = pd.Series(listy3)
se4 = pd.Series(listy4)
ceo_tweet['negative'] = se1.values
ceo_tweet['neutral'] = se2.values
ceo_tweet['positive'] = se3.values
ceo_tweet['compound'] = se4.values

pd.set_option('display.max_colwidth', -1)
#display full column

ceo_tweet.head(10)

,status_id,fulltext,negative,neutral,positive,compound
0,1115342985657495552,"""Advocates for human rights and free speech warn that the policies will lead to censorship.""\n\nThis is a given as governments are hampered by unfettered freedom of communications among the masses. That is where accountability begins. https://t.co/ZwxKOdzuuh",0.033,0.788,0.179,0.7964
1,1043171266008625152,"""Either consumers will pay more, suppliers will receive less, retail margins will be lower, or consumers will buy fewer products or forego purchases altogether.""\n\nOr all of the above, spreading the impact and mitigating any fallout. https://t.co/55r9cfuUp2",0.093,0.907,0.000,-0.3818
2,1059830148483166209,"""Every dollar invested in sanitation yields about $5.50 in global economic returns, according to the World Health Organization.""\n\nThat is a fascinating statistic, and one for those with conscious capitalist mindsets to take note of. https://t.co/fUTMz1ODXV",0.000,0.907,0.093,0.5423
3,1110242701692289024,"#DareToServe is a book for practitioners—people leading right now—in any organization, large or small, at any level. If you have been given a position to #lead people, it is for you. Pick up a copy today! https://t.co/5YzrJh1aw5 https://t.co/9A6k2bkxmZ",0.000,1.000,0.000,0.0000
4,1021866865935740928,"2008 was a seminal year for @Nasdaq. Following our merger with OMX, Nasdaq acquired the Philadelphia Stock Exchange (PHLX), the oldest stock exchange in the U.S. Today, we celebrate the 10-year anniversary of this historic occasion. https://t.co/uH2VXIWr8g",0.000,0.904,0.096,0.5719
5,1118750312838455297,"@BetoORourke has disqualified himself as a serious Presidential candidate when he called the leader of our key ally and only democracy in the Middle East, Israel’s @netanyahu a racist. Democratic Party can not take him seriously. https://t.co/cMzP7pmxok",0.191,0.709,0.100,-0.5970
6,1075458012838547456,@CesarFloresTV I'm sorry to hear about your experience and thank you for reaching out. Please work with our Corporate Customer Relations Team regarding this. You can either email them at Customer.Relations@AutoNation.com or call them at 954-769-3555. Mike.,0.032,0.807,0.161,0.6486
7,1112691118100099073,@HowardSchultz is by far the most qualified candidate in the race. Unless he gets tremendous financial and grass roots support as an Independent he should consider re entering as a Democrat. He could win the Democratic nomination. https://t.co/bdYeCzcQEk,0.000,0.843,0.157,0.7579
8,968518538859171845,@LouMM Here are some clues of companies only trying to survive. \n\n1) They are spending more time looking for money rather than running their business.\n2) They are not growing revenue or hiring.\n3) Founders are not taking a salary.\n4) They are out of cash.,0.033,0.967,0.000,-0.1326
9,936633471925215233,"@MageeRehab helps improve the quality of life of those with disabilities, and they do it with excellence, compassion and innovation. It’s an absolute honor to be involved with, and to be recognized by this inspiring organization. https://t.co/W0eFTPTgJt",0.000,0.581,0.419,0.9668


In [7]:
import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
ceo_tweet.to_gbq('ceo_tweet_streaming.sentiment', 'ra-work-content')

138995 out of 138995 rows loaded.


## note the data in 2017 has duplication due to the ceo may be in other cop other year
## there are 4 ceo duplicated recorded